### Finding power series expansions along exceptional components
------------------------------------------------------------------------------------------
First we define the surface, for now only in affine form. This is the surface Wittenberg works with in his article

In [ ]:
A3<x,y,t>:=AffineSpace(Rationals(),3);
p:=3*(t-1)^3*(t+3);
q:=Evaluate(p,[x,y,-t]);
F:=y^2-x*(x-p)*(x-q);
E:=Surface(A3,F);

We also must compute the equations for $E$ on the affine patch about infinity.

In [ ]:
p_inf:=3*(1-t)^3*(1+3*t);
q_inf:=3*(-1-t)^3*(-1+3*t);
F_inf:=y^2-x*(x-p_inf)*(x-q_inf);
E_inf:=Surface(A3, F_inf);

We check the discriminant of the elliptic curve. We need to do some hacky algebra to extract the polynomial we're interested in.

In [ ]:
Factorization(Discriminant(F-y^2,x));
Factorization(Discriminant(F_inf-y^2,x));

Singular subscheme, and some of the points in the support

In [ ]:
Support(SingularSubscheme(E));
Support(SingularSubscheme(E_inf));

In [ ]:
Etilde:=DesingulariseSurfaceByBlowUp(E);
Etilde_inf:=DesingulariseSurfaceByBlowUp(E_inf);

### Residue formula

Given a complete local DVR $R$ with uniformizer $\pi$ and residue field $k$, we compute the residue of a quaternion algebra $(f,g)$ as the class of $(-1)^{v(f)v(g)}\widetilde{f}^{v(g)}\widetilde{g}^{v(f)}$ in $k^*/k^{*2}$ where for any $h\in R$ we define $\widetilde{h}=h\pi^{-v(h)}$.

In [ ]:
function ComputeResidue(f, g)
    vf:=Valuation(f);
    vg:=Valuation(g);
    
    if Maximum([vg, vf]) gt 4 then
        printf "Valuation %o is too high", Maximum([vg,vf]);
    end if;
    
    f_0:=Coefficients(f)[1];
    g_0:=Coefficients(g)[1];
    
    return f_0^(vg)*g_0^(vf)*(-1)^(vf*vg);
end function;

function IsSquareQU(f)
    R := PolynomialRing(Integers());
    return IsSquare(R ! Numerator(f)) and IsSquare(R ! Denominator(f));
end function;

function ComputeAllInterestingResidues(X, T, IsInf)
    PT := 3*(T-1)^3*(T+3);
    QT := 3*(-T-1)^3*(-T+3);
    if IsInf then 
        PT := 3*(1/T-1)^3*(1/T+3);
        QT := 3*(-1/T-1)^3*(-1/T+3);
    end if;
    Glist := [X-PT, X-QT];
    Flist := [-1,2,3,T, T+1, T-1, T+3, T-3];
    if IsInf then
        Flist := [-1,2,3,1/T, 1/T+1, 1/T-1, 1/T+3, 1/T-3];
    end if;
    return [ [ ComputeResidue(f,g) : f in Flist ] : g in Glist];
end function;

### Compute Parametrization

In [ ]:
function GetRationalPoint(L)
    Lbar:=ProjectiveClosure(L);
    C,pi:=Conic(Lbar);
    P:=Inverse(pi)(RationalPoint(C));
    return P;
end function;

In [ ]:
function ComputeParametrization(L, P)
    kU<U>:=FunctionField(Rationals());
    Lc:=ProjectiveClosure(L);
    parm:=Eltseq(Lc(kU)![Evaluate(g,[U,1]): g in DefiningPolynomials(Parametrization(Lc,Lc!P))]);
    Xu:=parm[1]/parm[3];
    Yu:=parm[2]/parm[3];
    assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};
    return Xu, Yu;
end function;

## Computing list of residues for all bad fibers

### Computing residue at ((-9, 0), 0) 

#### Component 1

Here we are computing the residue along the original singular fiber $\mathcal{E}_0$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,0]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals());

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
Xu,Yu:=ComputeParametrization(L, [0, 1, 0]);

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu;

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
ResidueList:=[* *];
Append(~ResidueList,Residues);
Residues;

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((-9, 0),0)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[1],1);
A3<T,S,W>:=Ambient(Y);
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals());

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
A2<S0, W0>:=AffinePlane(Rationals());
L0:=Curve(A2, Evaluate(DefiningPolynomials(L)[1], [0,S0,W0]));
Su, Wu:=ComputeParametrization(L0, GetRationalPoint(L0));
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Su,Wu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu;

In [ ]:
R<Wpol>:=PolynomialRing(kUV);

Below we compute a higher order approximation of $W$ in the power series ring $k(U)[[V]]$.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Tu, Su, Wpol]);
Fprime:=Derivative(F);
for i in [1..3] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series in $k(U)[[V]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Tu+O(Vu^4),Su+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $T,S,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Tu, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Tu, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

### Computing residue at ((0, 0), -3) 

#### Component 1

Here we are computing the residue along the original singular fiber $\mathcal{E}_{-3}$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,-3]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals());

We write down a parametrization of L and check that it satisfies that parametrization.

In [ ]:
Xu,Yu:=ComputeParametrization(L, [0,1,0]);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu-3;

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), -3)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[2],1);
A3<X,S,W>:=Ambient(Y);
L;

Below we find a parametrization of $L$.

In [ ]:
A2<S0, W0>:=AffinePlane(Rationals());
L0:=Curve(A2, Evaluate(DefiningPolynomials(L)[1], [0,S0,W0]));
Su, Wu:=ComputeParametrization(L0, GetRationalPoint(L0));
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Su,Wu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Xu:=Vu;

Below we compute a higher order approximation of $W$ in the power series ring $k(U)[[V]]$.

In [ ]:
R<Wpol>:=PolynomialRing(kUV);
F:=Evaluate(DefiningPolynomial(Y), [Xu, Su, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series in $k(U)[[V]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu+O(Vu^4),Su+O(Vu^4),Wu+O(Vu^4)]);

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

### Computing residues at ((0, 0), -1) 

#### Component 1

Here we compute the residue at the original component of $\mathcal{E}_{-1}$

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,-1]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals());

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
Xu,Yu:=ComputeParametrization(L, [0,1,0]);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu-1;

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), -1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[3],1);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<X,S,W>:=Ambient(Y);
L;

Note that $L:S^2-48=0, X=0$ has no rational points and is geometrically reducible. In particular, one can work out that the residue field is $\kappa(L)=k(W)$ where $k=\mathbb{Q}(a)$ and $a$ is a square root of $3$. Below we define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(48);
kU<U>:=FunctionField(k);
Wu:=U;

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Xu:=Vu;

In [ ]:
Q<Spol>:=PolynomialRing(kUV);

By definition we have an expression for $X$ and $W$ in $k(U)(Vu)$ as $W=U$ and $X=Vu$. It remains to find an expansion for $S$. From our observations above we note that when taking residues $\overline{S}=4a=4\sqrt{3}\in k(W)$, as we mod out by $S^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Spol, Wu]);
Fprime:=Derivative(F);
Su:=4*a;
for i in [1..4] do
    Temp:=Su-Evaluate(F, Su)/Evaluate(Fprime,Su);
    Su:=Temp;
end for;

We verify that the power series $Xu,Su,Wu$ in $k(U)[[Vu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu+O(Vu^4),Su+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,S,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

#### Component 3

Here we unpack the equations for the second exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), -1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[3],2);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<R,S,W>:=Ambient(Y);
L;

Note that $L:S^2-48=0, R=0$ has no rational points and is geometrically reducible. In particular, one can work out that the residue field is $\kappa(L)=k(W)$ where $k=\mathbb{Q}(a)$ and $a$ is a square root of $3$. Below we define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(48);
kU<U>:=FunctionField(k);
Su:=U;

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Ru:=Vu;

In [ ]:
Q<Wpol>:=PolynomialRing(kUV);

By definition we have an expression for $S$ and $W$ in $k(U)(Vu)$ as $S=U$ and $R=Vu$. It remains to find an expansion for $W$. From our observations above we note that when taking residues $\overline{W}=4a=4\sqrt{3}\in k(W)$, as we mod out by $W^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Ru, Su, Wpol]);
Fprime:=Derivative(F);
Wu:=4*a;
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Ru,Su,Wu$ in $k(U)[[Vu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Ru+O(Vu^4),Su+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on $\mathcal{E}$, we use the description of pi, that expresses $X,Y,T$ in terms of $R,S,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Ru, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Ru, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

#### Component 4

Here we unpack the equations for the third exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), -1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[3],3);

In [ ]:
A3<T,S,W>:=Ambient(Y);
L;

We write down a parametrization of our conic $L$. For the parametrization below we took the projective closure and projected away from the rational point $[0,0,1]$.

In [ ]:
kU<U>:=FunctionField(Rationals());
A2<S0, W0>:=AffinePlane(Rationals());
L0:=Curve(A2, Evaluate(DefiningPolynomials(L)[1], [-1,S0,W0]));
Su, Wu:=ComputeParametrization(L0, GetRationalPoint(L0));
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[-1,Su,Wu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu-1;
R<Wpol>:=PolynomialRing(kUV);

Now we compute an expansion for $W$ in $\mathbb{Q}(U)[[T]]$.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Tu, Su, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Tu,Vu,Wu$ in $k(U)[[Tu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Tu+O(Vu^4),Su+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Tu, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Tu, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

### Computing residues at ((0, 0), 1) 

#### Component 1

Here we compute the residues over the original component $\mathcal{E}_1$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,1]));

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals());
Xu,Yu:=ComputeParametrization(L, [0,1,0]);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu+1;

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[4],1);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<X,S,W>:=Ambient(Y);
L;

Note that $L:V^2-48=0, X=0$ has no rational points and is geometrically reducible. In particular, one can work out that the residue field is $\kappa(L)=k(W)$ where $k=\mathbb{Q}(a)$ and $a$ is a square root of $3$. Below we define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(48);
kU<U>:=FunctionField(k);
Wu:=U;

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Xu:=Vu;

In [ ]:
Q<Spol>:=PolynomialRing(kUV);

By definition we have an expression for $X$ and $W$ in $k(U)(Xu)$ as $W=U$ and $X=Xu$. It remains to find an expansion for $S$. From our observations above we note that when taking residues $\overline{S}=4a=4\sqrt{3}\in k(W)$, as we mod out by $S^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Spol, Wu]);
Fprime:=Derivative(F);
Su:=4*a;
for i in [1..4] do
    Temp:=Su-Evaluate(F, Su)/Evaluate(Fprime,Su);
    Su:=Temp;
end for;

We verify that the power series $Xu,Su,Wu$ in $k(U)[[Vu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu+O(Vu^4),Su+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

#### Component 3

Here we unpack the equations for the second exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[4],2);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<R,S,W>:=Ambient(Y);
L;

We define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(48);
kU<U>:=FunctionField(k);
Su:=U;

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Ru:=Vu;

In [ ]:
Q<Wpol>:=PolynomialRing(kUV);

By definition we have an expression for $R$ and $S$ in $k(U)(Xu)$ as $S=U$ and $R=Vu$. It remains to find an expansion for $W$. We note that when taking residues we have $\overline{W}=4a=4\sqrt{3}$, as we mod out by $W^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Ru, Su, Wpol]);
Fprime:=Derivative(F);
Wu:=4*a;
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Ru,Su,Wu$ in $k(U)[[Vu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Ru+O(Vu^4),Su+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $R,S,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Ru, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Ru, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

#### Component 4

Here we unpack the equations for the third exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[4],3);

In [ ]:
A3<T,S,W>:=Ambient(Y);
L;

We write down a parametrization of our conic $L$.

In [ ]:
kU<U>:=FunctionField(Rationals());
A2<S0, W0>:=AffinePlane(Rationals());
L0:=Curve(A2, Evaluate(DefiningPolynomials(L)[1], [1,S0,W0]));
Su, Wu:=ComputeParametrization(L0, GetRationalPoint(L0));
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[1,Su,Wu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu+1;

In [ ]:
Q<Wpol>:=PolynomialRing(kUV);

Now we compute an expansion for $W$ in $\mathbb{Q}(U)[[V]]$.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Tu, Su, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Tu,Su,Wu$ in $k(U)[[Vu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Tu+O(Vu^4),Su+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Tu, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Tu, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

### Computing residue at ((0, 0), 3) 

#### Component 1

Here we compute the residues over the original component $\mathcal{E}_3$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E), [x2,y2,3]));

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals());
Xu,Yu:=ComputeParametrization(L, [0,1,0]);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Tu:=Vu+3;

In [ ]:
Residues:=ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 3)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[5],1);
A3<X,S,W>:=Ambient(Y);
L;

Below we find a parametrization of $L$.

In [ ]:
kU<U>:=FunctionField(Rationals());
A2<S0, W0>:=AffinePlane(Rationals());
L0:=Curve(A2, Evaluate(DefiningPolynomials(L)[1], [0,S0,W0]));
Su, Wu:=ComputeParametrization(L0, GetRationalPoint(L0));
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Su,Wu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Xu:=Vu;

In [ ]:
Q<Wpol>:=PolynomialRing(kUV);

Here we compute a higher order approximation for $W$ in the power series ring.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Su, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series in $k(U)[[V]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu+O(Vu^4),Su+O(Vu^4),Wu+O(Vu^4)]);

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Su, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Su, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Tu, false);
Append(~ResidueList,Residues);

### Computing residues at ((3, 0), $\infty$) 

#### Component 1

Here we compute the residues over the original component $\mathcal{E}_{\infty}$.

In [ ]:
A2<x2,y2>:=AffinePlane(Rationals());
L:=Curve(A2, Evaluate(DefiningPolynomial(E_inf), [x2,y2,0]));
L;

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals());
Xu,Yu:=ComputeParametrization(L, [0,1,0]);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[Xu,Yu]) eq 0};

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Su:=Vu;

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Su, true);
Append(~ResidueList,Residues);

#### Component 2

Here we unpack the equations for the first exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), \infty)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E_inf,Etilde_inf[1],1);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<S,R,W>:=Ambient(Y);
L;

The variable $S$ corresponds to $1/t$ in our original coordinates.

Note that $L:R^2-3=0, S=0$ has no rational points and is geometrically reducible. In particular, one can work out that the residue field is $\kappa(L)=k(W)$ where $k=\mathbb{Q}(a)$ and $a$ is a square root of $3$. Below we define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(3);
kU<U>:=FunctionField(k);
Wu:=U;

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Su:=Vu;

In [ ]:
Q<Rpol>:=PolynomialRing(kUV);

By definition we have an expression for $S$ and $W$ in $k(U)(Vu)$ as $W=U$ and $S=Vu$. It remains to find an expansion for $R$. From our observations above we note that when taking residues $\overline{R}=a=\sqrt{3}\in k(W)$, as we mod out by $R^2-3$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Su, Rpol, Wu]);
Fprime:=Derivative(F);
Ru:=a;
for i in [1..4] do
    Temp:=Ru-Evaluate(F, Ru)/Evaluate(Fprime,Ru);
    Ru:=Temp;
end for;

We verify that the power series $Su,Ru,Wu$ in $k(U)[[Vu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Su+O(Vu^4),Ru+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on $E$, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Su, Ru, Wu]);
Su:=Evaluate(DefiningPolynomials(pi)[3], [Su, Ru, Wu]);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Su, true);
Append(~ResidueList,Residues);

#### Patch 3

Here we unpack the equations for the second exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E_inf,Etilde_inf[1],2);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<P,R,W>:=Ambient(Y);
L;

Here we see that $L:W^2/972-(R+1/18)^2=0$ and therefore $L$ reducible to the product of two lines over $\mathbb{Q}(\sqrt{972})=\mathbb{Q}(\sqrt{3})$. Thus the residue field $\kappa(L)$ is $\mathbb{Q}(\sqrt{3})(U)$ with $U=R$.

In [ ]:
k<a>:=QuadraticField(3);
kU<U>:=FunctionField(k);
Ru:=U;
Wu:=(18*Ru+1)*a;

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Pu:=Vu;

In [ ]:
Q<Wpol>:=PolynomialRing(kUV);

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Pu, Ru, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Pu,Ru,Wu$ in $k(U)[[Vu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Pu+O(Vu^4),Ru+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on $E$, we use the description of pi, that expresses $X,Y,T$ in terms of $P,R,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Pu, Ru, Wu]);
Su:=Evaluate(DefiningPolynomials(pi)[3], [Pu, Ru, Wu]);

In [ ]:
Residues:=ComputeAllInterestingResidues(Xu, Su, true);
Append(~ResidueList,Residues);

#### Component 4

Here we unpack the equations for the third exceptional component L on a patch Y of the desingularization $\tilde{\mathcal{E}}$ of $\mathcal{E}$ over $((0,0), 1)$, together with the blowdown morphism $\pi: Y \to \mathcal{E}$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E_inf,Etilde_inf[1],3);

In [ ]:
A3<S,R,W>:=Ambient(Y);
L;

We write down a parametrization of our conic $L$.

In [ ]:
kU<U>:=FunctionField(Rationals());
A2<R0, W0>:=AffinePlane(Rationals());
L0:=Curve(A2, Evaluate(DefiningPolynomials(L)[1], [0,R0,W0]));
Ru, Wu:=ComputeParametrization(L0, GetRationalPoint(L0));
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Ru,Wu]) eq 0};

In [ ]:
kUV<Vu>:=PowerSeriesRing(kU);
Su:=Vu;

In [ ]:
Q<Wpol>:=PolynomialRing(kUV);

Now we compute an expansion for $W$ in $\mathbb{Q}(U)[[Vu]]$.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Su, Ru, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Su,Ru,Wu$ in $k(U)[[Vu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Su+O(Vu^4),Ru+O(Vu^4),Wu+O(Vu^4)]);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,S$ in terms of $S,R,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Su, Ru, Wu]);
Su:=Evaluate(DefiningPolynomials(pi)[3], [Su, Ru, Wu]);
Xu+O(Vu^4);
Su+O(Vu^4);

In [ ]:
Residues := ComputeAllInterestingResidues(Xu, Su, true);
Append(~ResidueList,Residues);
Residues;

**Working on analyzing the residues**

In [ ]:
W:=[];
for Vpair in ResidueList do
    V:=&cat Vpair;
    FnFactorBasis:={a[1]: a in &cat[Factorization(Numerator(v)): v in V] cat &cat[Factorization(Denominator(v)): v in V]};
    KK:=Universe(V);
    Kc:=BaseRing(KK);
    OK:=IntegerRing(Kc);
    if OK cmpeq IntegerRing() then
      S:=PrimeFactors(6*&*[Numerator(c)*Denominator(c) where c:=LeadingCoefficient(Numerator(a)): a in V]);
      arch_modsquares:=func< c | [Sign(c) eq 1 select 0 else 1]>;
    else
      S:=Support(6*&*[Numerator(c)*Denominator(c) where c:=LeadingCoefficient(Numerator(a)): a in V]*OK);
      arch_modsquares:=func< c | [s eq 1 select 0 else 1 : s in RealSigns(c)]>;
    end if;
    selmaps:=[* arch_modsquares *];
    for p in S do
       Kc_complete,toKc:=Completion(Kc,p);
       _,toSel:=pSelmerGroup(2,Kc_complete);
       Append(~selmaps,func<c | Eltseq(toSel(toKc(c)))>);
    end for;
    
    //Append(~W,[
    //    [Valuation(v,p): p in FnFactorBasis] cat &cat[s(LeadingCoefficient(Numerator(v))):s in selmaps]:v in V]);
    Append(~W,[[Valuation(v,p): p in FnFactorBasis]:v in V]);
end for;

In [ ]:
M:=Matrix(GF(2),[&cat[w[i]: w in W]: i in [1..#W[1]]]);
bs:=Kernel(M);

In [ ]:
WW:=Matrix(GF(2),[[0,1,1,1,1,0,0,0,0,1,1,1,0,1,0,0]]);
WW*M;

In [ ]:
vectoalg:=func<b|<&*[Parent(T)|[-1,2,3,T, T+1, T-1, T+3, T-3, -1,2,3,T, T+1, T-1, T+3, T-3][i]: i in [1..8] | b[i] eq 1],
  &*[Parent(T)|[-1,2,3,T, T+1, T-1, T+3, T-3, -1,2,3,T, T+1, T-1, T+3, T-3][i]: i in [9..16] | b[i] eq 1]>>;

In [ ]:
imgx0:=([1,0,1,0,1,0,0,1, 1,0,1,0,0,1,1,0]); // 2-torsion
imgx1:= [0,0,1,1,0,0,0,0, 0,0,0,1,0,1,1,0];  // 2-torsion
imgB:= [0,0,0,1,0,0,0,0, 0,0,0,1,0,0,0,0];
imgA:= [0,1,1,1,1,0,0,0,0,1,1,1,0,1,0,0];

imgm1z:=[1,0,0,0,0,0,0,0, 0,0,0,0,0,0,0,0];
img2z:=[0,1,0,0,0,0,0,0, 0,0,0,0,0,0,0,0];
img3z:=[0,0,1,0,0,0,0,0, 0,0,0,0,0,0,0,0];
imgzm1:=[0,0,0,0,0,0,0,0, 1,0,0,0,0,0,0,0];
imgz2:=[0,0,0,0,0,0,0,0, 0,1,0,0,0,0,0,0];
imgz3:=[0,0,0,0,0,0,0,0, 0,0,1,0,0,0,0,0];

In [ ]:
bs;

In [ ]:
sub<bs|imgx0,imgx1,imgB,imgA,imgm1z,img2z,img3z,imgzm1,imgz2,imgz3>;

In [ ]:
Factor(vectoalg(imgx1)[2]);

In [ ]:
[r[2][6]: r in ResidueList];

In [ ]:
[Factor(r[1][4]*r[2][4]): r in ResidueList];